In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd



In [ ]:

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
dataset = ImageFolder(r"C:\Users\manoj\OneDrive\Desktop\Program Files\Python\Intership1\task3\train", transform=transforms.ToTensor())
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)



In [ ]:
# Define feature extractor using MaxPooling and Flatten
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Reduce dimensions
        self.flatten = nn.Flatten()
    
    def forward(self, x):
        x = self.pool(x)  # Apply max pooling
        return self.flatten(x)


In [ ]:

# Initialize feature extractor and move to device
extractor = FeatureExtractor().to(device)

# Extract features and labels
features, labels = [], []
for images, targets in dataloader:
    images = images.to(device)  # Move images to GPU if available
    extracted_features = extractor(images).cpu().numpy()  # Move back to CPU for NumPy processing
    features.append(extracted_features)
    labels.append(targets.numpy())

features, labels = np.vstack(features), np.hstack(labels)

# Create DataFrame
data = pd.DataFrame(features)
data['target'] = labels



In [ ]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train SVM model
svm = SVC(kernel='linear').fit(X_train, y_train)

# Predict and generate classification report
predictions = svm.predict(X_test)
print(classification_report(y_test, predictions))